# Actual Implementation:
### Based on _"Parallel feedback processing for voluntary control: knowledge of spinal feedback in motor cortex"_

### **1. Understand the Paper's Model Components**
The paper describes a hierarchical control model with the following components:
- **Spinal Reflex Pathway**: Includes stretch reflex dynamics (already implemented in your model).
- **Recurrent Neural Networks (RNNs)**: Represent cortical feedback pathways (sensory and motor cortices).
- **Musculoskeletal Model**: A two-segment planar arm with six muscles (monoarticular and biarticular).
- **Proprioceptive Feedback**: Includes joint angles, velocities, and muscle states.
- **Training Objective**: Optimize the RNN to minimize energy consumption and kinematic error during a posture perturbation task.

### **2. Plan the Implementation**

##### **Step 1: Extend the Musculoskeletal Model**
- Replace the single-joint model with a **two-segment planar arm**.
  - **Segments**: Humerus and forearm.
  - **Joints**: Shoulder and elbow.
  - **Muscles**: Four monoarticular (shoulder flexor/extensor, elbow flexor/extensor) and two biarticular (biceps and triceps).
- Implement muscle kinematics:
  - Muscle lengths and velocities depend on joint angles and moment arms.
  - Use the equations provided in the paper for muscle length and velocity.
- Add muscle force dynamics:
  - Include force-length and force-velocity relationships for each muscle.

##### **Step 2: Add the RNN Controller**
- Implement a **two-layer RNN**:
  - **Sensory Layer**: Processes proprioceptive feedback (joint angles, velocities, muscle forces).
  - **Motor Layer**: Generates descending commands to muscles.
- Define the RNN dynamics:
  - Use the discretized neural activation equation provided in the paper.
  - Include time constants, biases, and weight matrices for each layer.
- Train the RNN:
  - Use the Adam optimizer to minimize the composite cost function (energy + kinematic error).
  - Define the cost function as described in the paper.

##### **Step 3: Integrate Spinal Reflexes**
- Combine the RNN output with spinal reflex contributions to compute muscle activation.
- Include the **spinal reflex delay** (20 ms) and **afferent encoding**:
  - Encode muscle kinematics (length, velocity, acceleration, jerk) into afferent signals.
  - Scale the spinal reflex gain for high and low background load conditions.

##### **Step 4: Simulate the Posture Perturbation Task**
- Implement the task setup:
  - Apply background loads (low and high) to the shoulder and elbow joints.
  - Introduce a step perturbation load after a random delay.
- Simulate the task:
  - Use the Euler integration method with a time step of 10 ms.
  - Record joint angles, velocities, muscle activations, and neural activity.

##### **Step 5: Analyze and Compare Results**
- Compare the simulation results with the experimental data:
  - **Muscle EMG**: Verify gain scaling in the spinal reflex response.
  - **Neural Activity**: Check for reduced cortical activity in high background load conditions.
  - **Kinematics**: Analyze joint displacement and hand trajectories.
- Perform PCA on neural activity to visualize trajectories in neural space.

### **3. Implementation Details**

##### **Musculoskeletal Model**
- Use the Lagrange equation to compute joint torques and motion.
- Define the moment arms for each muscle across the shoulder and elbow joints.
- Implement the muscle force-length and force-velocity properties.

##### **RNN Dynamics**
- Define the sensory and motor layers with 200 units each.
- Initialize weights and biases randomly.
- Use delayed proprioceptive feedback as input to the sensory layer.
- Project the motor layer output to the muscle activation layer.

##### **Training**
- Train the RNN using 16 load conditions (8 directions × 2 load sizes).
- Use the composite cost function to optimize the network.

##### **Simulation**
- Simulate the task for 3000 ms:
  - Background load: 0-1500 ms.
  - Perturbation load: 1500-3000 ms.
- Record neural and kinematic responses.

### **4. Tools and Libraries**
- **Python**: Use NumPy, SciPy, and PyTorch for numerical computations and RNN training.
- **Visualization**: Use Matplotlib for plotting kinematics, EMG, and neural activity.
- **Simulation**: Use `solve_ivp` or a custom Euler integrator for the musculoskeletal dynamics.

### **5. Validation**
- Compare your results with the figures in the paper:
  - Ensure the spinal reflex gain scaling matches the experimental data.
  - Verify the reduction in cortical activity for high background loads.
  - Reproduce the neural trajectories using PCA.

### **6. Next Steps**
- Once the model is validated, explore extensions:
  - Test the model with different perturbation profiles.
  - Investigate the role of cortical feedback in adapting to novel tasks.

---

### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
import torch.optim as optim

### Define the musculoskeletal model

In [2]:
class MusculoskeletalModel:
    def __init__(self, moment_arms, muscle_params):
        """
        Initialize the musculoskeletal model.
        Args:
            moment_arms: Dictionary of moment arms for each muscle.
            muscle_params: Dictionary of muscle parameters (e.g., Fmax, Lopt, etc.).
        """
        self.moment_arms = moment_arms
        self.muscle_params = muscle_params

    def muscle_length_velocity(self, joint_angles, joint_velocities):
        """
        Compute muscle lengths and velocities based on joint angles and velocities.
        Args:
            joint_angles: Array of joint angles [shoulder, elbow].
            joint_velocities: Array of joint angular velocities [shoulder, elbow].
        Returns:
            lengths: Muscle lengths.
            velocities: Muscle velocities.
        """
        lengths = {}
        velocities = {}
        for muscle, params in self.muscle_params.items():
            r = self.moment_arms[muscle]
            lengths[muscle] = params['L0'] - np.dot(r, joint_angles)
            velocities[muscle] = -np.dot(r, joint_velocities)
        return lengths, velocities

    def muscle_force(self, activations, lengths, velocities):
        """
        Compute muscle forces based on activations, lengths, and velocities.
        Args:
            activations: Muscle activations.
            lengths: Muscle lengths.
            velocities: Muscle velocities.
        Returns:
            forces: Muscle forces.
        """
        forces = {}
        for muscle, activation in activations.items():
            params = self.muscle_params[muscle]
            fL = np.exp(-((lengths[muscle] - params['Lopt']) / params['wL'])**2)
            fV = 1 / (1 + params['cV'] * velocities[muscle])
            forces[muscle] = params['Fmax'] * activation * fL * fV
        return forces

    def joint_torques(self, forces):
        """
        Compute joint torques based on muscle forces.
        Args:
            forces: Muscle forces.
        Returns:
            torques: Joint torques [shoulder, elbow].
        """
        torques = np.zeros(2)
        for muscle, force in forces.items():
            r = self.moment_arms[muscle]
            torques += r * force
        return torques

### Define the spinal reflex pathway

In [3]:
class SpinalReflex:
    def __init__(self, reflex_gains, delay=0.02):
        """
        Initialize the spinal reflex pathway.
        Args:
            reflex_gains: Dictionary of reflex gains for each muscle.
            delay: Reflex delay (seconds).
        """
        self.reflex_gains = reflex_gains
        self.delay = delay

    def compute_activation(self, muscle_states):
        """
        Compute spinal reflex activation based on muscle states.
        Args:
            muscle_states: Dictionary of muscle states (length, velocity, etc.).
        Returns:
            activations: Reflex activations for each muscle.
        """
        activations = {}
        for muscle, state in muscle_states.items():
            length, velocity = state
            activation = self.reflex_gains[muscle] * (length + velocity)
            activations[muscle] = max(0, activation)  # Ensure non-negative activation
        return activations

### Define the RNN controller

In [4]:
class RNNController(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        Initialize the RNN controller.
        Args:
            input_size: Number of input features.
            hidden_size: Number of hidden units.
            output_size: Number of output features.
        """
        super(RNNController, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        """
        Forward pass through the RNN.
        Args:
            x: Input tensor.
            h: Hidden state tensor.
        Returns:
            output: Output tensor.
            h: Updated hidden state tensor.
        """
        out, h = self.rnn(x, h)
        out = self.fc(out)
        return out, h

### Define the training loop for the RNN

In [5]:
def train_rnn(controller, data_loader, num_epochs, learning_rate):
    """
    Train the RNN controller.
    Args:
        controller: RNNController instance.
        data_loader: DataLoader for training data.
        num_epochs: Number of training epochs.
        learning_rate: Learning rate for the optimizer.
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(controller.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for inputs, targets in data_loader:
            h = torch.zeros(1, inputs.size(0), controller.rnn.hidden_size)
            outputs, _ = controller(inputs, h)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

### Simulate the posture perturbation task

In [6]:
def simulate_task(muscle_model, reflex, controller, task_params):
    """
    Simulate the posture perturbation task.
    Args:
        muscle_model: MusculoskeletalModel instance.
        reflex: SpinalReflex instance.
        controller: RNNController instance.
        task_params: Dictionary of task parameters (e.g., loads, durations).
    Returns:
        results: Simulation results (e.g., joint angles, velocities, activations).
    """
    # Initialize state variables
    joint_angles = np.zeros(2)
    joint_velocities = np.zeros(2)
    muscle_activations = {muscle: 0.05 for muscle in muscle_model.muscle_params}

    results = {'angles': [], 'velocities': [], 'activations': []}

    # Simulate over time
    for t in np.arange(0, task_params['duration'], task_params['dt']):
        # Compute muscle states
        lengths, velocities = muscle_model.muscle_length_velocity(joint_angles, joint_velocities)
        muscle_states = {m: (lengths[m], velocities[m]) for m in lengths}

        # Compute spinal reflex activations
        reflex_activations = reflex.compute_activation(muscle_states)

        # Compute RNN controller output
        proprioceptive_feedback = np.hstack([joint_angles, joint_velocities])
        proprioceptive_feedback = torch.tensor(proprioceptive_feedback, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        h = torch.zeros(1, 1, controller.rnn.hidden_size)
        rnn_output, _ = controller(proprioceptive_feedback, h)
        rnn_activations = rnn_output.squeeze(0).detach().numpy()

        # Combine reflex and RNN activations
        muscle_activations = {m: reflex_activations[m] + rnn_activations[i]
                              for i, m in enumerate(muscle_activations)}

        # Compute muscle forces and joint torques
        forces = muscle_model.muscle_force(muscle_activations, lengths, velocities)
        torques = muscle_model.joint_torques(forces)

        # Update joint dynamics
        joint_accelerations = torques / task_params['inertia']
        joint_velocities += joint_accelerations * task_params['dt']
        joint_angles += joint_velocities * task_params['dt']

        # Store results
        results['angles'].append(joint_angles.copy())
        results['velocities'].append(joint_velocities.copy())
        results['activations'].append(muscle_activations.copy())

    return results

### Visualize results

In [7]:
def plot_results(results):
    """
    Plot the simulation results.
    Args:
        results: Simulation results (e.g., joint angles, velocities, activations).
    """
    t = np.arange(len(results['angles']))
    angles = np.array(results['angles'])
    velocities = np.array(results['velocities'])

    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.plot(t, angles[:, 0], label='Shoulder Angle')
    plt.plot(t, angles[:, 1], label='Elbow Angle')
    plt.xlabel('Time (ms)')
    plt.ylabel('Angle (rad)')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(t, velocities[:, 0], label='Shoulder Velocity')
    plt.plot(t, velocities[:, 1], label='Elbow Velocity')
    plt.xlabel('Time (ms)')
    plt.ylabel('Velocity (rad/s)')
    plt.legend()

    plt.tight_layout()
    plt.show()